# 第四课：使用 LangGraph 构建智能体

## 学习目标
1. 理解 LangGraph 的核心概念
2. 理解智能体的工作原理
3. 构建一个能够自主决策的智能体

## 什么是 LangGraph？

LangGraph 是一个用于构建有状态、多步骤 AI 应用的库：
- 使用**图（Graph）**来表示工作流
- 支持**循环**和**条件判断**
- 非常适合构建智能体

## 什么是智能体（Agent）？

```
智能体 = 大语言模型 + 工具 + 自主决策能力
```

智能体可以：
- 根据任务**自主选择**使用哪些工具
- 在多个步骤中**持续工作**，直到完成任务
- **推理**和**规划**复杂任务

## 准备工作

In [ ]:
# 安装 LangGraph（如果还没安装）
!pip install langgraph -q

In [ ]:
import os
from datetime import datetime
from typing import TypedDict, Annotated, Sequence
import operator

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

# LangGraph 核心组件
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

# 设置 API Key
os.environ["DASHSCOPE_API_KEY"] = "your_api_key_here"

def create_qwen_chat():
    api_key = os.getenv("DASHSCOPE_API_KEY")
    if not api_key or api_key == "your_api_key_here":
        raise ValueError("⚠️ 请设置你的 DASHSCOPE_API_KEY")
    
    return ChatOpenAI(
        model="qwen-plus",
        openai_api_key=api_key,
        openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        temperature=0,
    )

print("✅ 准备完成")

---
## 1. 定义工具

首先定义智能体可以使用的工具。

In [ ]:
@tool
def get_current_time() -> str:
    """获取当前的日期和时间。"""
    return datetime.now().strftime("%Y年%m月%d日 %H:%M:%S")


@tool
def calculate(expression: str) -> str:
    """计算数学表达式，支持加减乘除和幂运算。"""
    try:
        allowed_chars = set("0123456789+-*/.() ")
        if not all(c in allowed_chars for c in expression):
            return "错误：表达式包含不允许的字符"
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"计算错误：{str(e)}"


@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息。"""
    weather_data = {
        "北京": {"temp": 15, "condition": "晴", "humidity": 45},
        "上海": {"temp": 18, "condition": "多云", "humidity": 60},
        "广州": {"temp": 25, "condition": "小雨", "humidity": 80},
        "深圳": {"temp": 26, "condition": "阴", "humidity": 75},
    }
    
    if city in weather_data:
        data = weather_data[city]
        return f"{city}天气：{data['condition']}，温度 {data['temp']}°C，湿度 {data['humidity']}%"
    return f"暂无 {city} 的天气数据"


@tool
def search_web(query: str) -> str:
    """搜索网络获取信息（模拟）。"""
    results = {
        "python": "Python 是最流行的编程语言之一，特别适合 AI 和数据科学。",
        "langchain": "LangChain 是构建 LLM 应用的流行框架，提供丰富的工具和抽象。",
        "langgraph": "LangGraph 用于构建有状态的多步骤 AI 工作流和智能体。",
    }
    
    for key, value in results.items():
        if key.lower() in query.lower():
            return value
    return f"搜索 '{query}' 未找到相关结果。"


# 工具列表
tools = [get_current_time, calculate, get_weather, search_web]
print(f"✅ 已定义 {len(tools)} 个工具")

---
## 2. 定义状态 (State)

**状态**是 LangGraph 的核心概念：
- 状态在图的各个节点之间传递
- 每个节点可以读取和修改状态
- 使用 `TypedDict` 定义状态结构

In [ ]:
class AgentState(TypedDict):
    """
    智能体状态
    
    messages: 消息列表
        - 使用 Annotated 和 operator.add 表示新消息追加到列表末尾
        - 而不是替换整个列表
    """
    messages: Annotated[
        Sequence[HumanMessage | AIMessage | ToolMessage],
        operator.add  # 新消息追加而非替换
    ]

print("✅ 状态定义完成")
print("\n状态结构:")
print("  messages: 消息列表 (追加模式)")

---
## 3. 构建智能体图

智能体图的结构：

```
        ┌──────────┐
        │  START   │
        └────┬─────┘
             │
             ▼
        ┌──────────┐
    ┌──▶│  agent   │◀──┐
    │   └────┬─────┘   │
    │        │         │
    │        ▼         │
    │   ┌──────────┐   │
    │   │ 条件判断  │   │
    │   └────┬─────┘   │
    │        │         │
    │   ┌────┴────┐    │
    │   │         │    │
    │   ▼         ▼    │
    │ ┌───┐   ┌──────┐ │
    │ │END│   │tools │─┘
    │ └───┘   └──────┘
```

In [ ]:
# 准备模型
model = create_qwen_chat()
model_with_tools = model.bind_tools(tools)

# 系统提示词
system_message = SystemMessage(content="""你是一个智能助手，可以使用工具来帮助用户解决问题。

你可以使用的工具：
- get_current_time: 获取当前时间
- calculate: 进行数学计算
- get_weather: 查询天气
- search_web: 搜索网络

请根据用户的问题，决定是否需要使用工具。回答要准确、简洁。""")

print("✅ 模型准备完成")

In [ ]:
# 定义 agent 节点
def agent_node(state: AgentState) -> dict:
    """
    智能体节点：调用 LLM 进行推理
    
    输入：当前状态
    输出：新的消息（追加到 messages）
    """
    # 在消息列表开头添加系统消息
    messages = [system_message] + list(state["messages"])
    
    # 调用模型
    response = model_with_tools.invoke(messages)
    
    # 返回新消息
    return {"messages": [response]}


# 定义条件函数
def should_continue(state: AgentState) -> str:
    """
    条件函数：检查是否需要继续执行工具
    
    返回值：
    - "tools": 需要执行工具
    - "end": 结束
    """
    last_message = state["messages"][-1]
    
    # 如果 AI 请求调用工具，则继续到 tools 节点
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    
    # 否则结束
    return "end"


print("✅ 节点函数定义完成")

In [ ]:
# 创建工具节点（LangGraph 提供的预构建节点）
tool_node = ToolNode(tools)

# 构建图
workflow = StateGraph(AgentState)

# 添加节点
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# 设置入口点
workflow.set_entry_point("agent")

# 添加条件边
workflow.add_conditional_edges(
    "agent",  # 从 agent 节点出发
    should_continue,  # 使用 should_continue 函数决定下一步
    {
        "tools": "tools",  # 如果返回 "tools"，去 tools 节点
        "end": END,  # 如果返回 "end"，结束
    }
)

# tools 执行完后，回到 agent
workflow.add_edge("tools", "agent")

# 编译图
agent = workflow.compile()

print("✅ 智能体构建完成！")
print("\n图的结构:")
print("  START → agent → [条件判断]")
print("                     ├→ tools → agent (循环)")
print("                     └→ END")

---
## 4. 运行智能体

In [ ]:
def run_agent(query: str):
    """运行智能体并显示执行过程"""
    print(f"\n{'='*50}")
    print(f"👤 用户: {query}")
    print(f"{'='*50}")
    
    # 运行智能体
    result = agent.invoke({
        "messages": [HumanMessage(content=query)]
    })
    
    # 打印执行过程
    print("\n📋 执行过程:")
    for i, msg in enumerate(result["messages"]):
        if isinstance(msg, HumanMessage):
            print(f"  [{i}] 👤 用户: {msg.content}")
        elif isinstance(msg, AIMessage):
            if msg.tool_calls:
                tools_called = [tc['name'] for tc in msg.tool_calls]
                print(f"  [{i}] 🤖 AI 调用工具: {tools_called}")
            elif msg.content:
                content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
                print(f"  [{i}] 🤖 AI 回复: {content}")
        elif isinstance(msg, ToolMessage):
            print(f"  [{i}] 🔧 工具结果: {msg.content}")
    
    # 最终回答
    final_answer = result["messages"][-1].content
    print(f"\n💬 最终回答: {final_answer}")
    
    return result

In [ ]:
# 测试：简单问候（不需要工具）
run_agent("你好！")

In [ ]:
# 测试：需要时间工具
run_agent("现在几点了？")

In [ ]:
# 测试：需要计算工具
run_agent("帮我计算 123 * 456 + 789")

In [ ]:
# 测试：需要天气工具
run_agent("北京今天天气怎么样？")

In [ ]:
# 测试：比较任务（可能需要多次工具调用）
run_agent("北京和上海今天哪个城市更热？")

In [ ]:
# 测试：复杂任务
run_agent("先告诉我现在的时间，然后查一下深圳的天气")

---
## 5. 流式输出

使用 `stream` 方法可以实时看到智能体的执行过程。

In [ ]:
query = "帮我查一下北京天气，然后告诉我现在的时间"
print(f"👤 用户: {query}")
print("\n📋 执行过程 (流式):")

# 使用 stream 方法
for event in agent.stream({"messages": [HumanMessage(content=query)]}):
    # event 是一个字典，键是节点名，值是该节点的输出
    for node_name, node_output in event.items():
        print(f"\n--- {node_name} 节点 ---")
        if "messages" in node_output:
            for msg in node_output["messages"]:
                if isinstance(msg, AIMessage):
                    if msg.tool_calls:
                        print(f"  🤖 调用: {[tc['name'] for tc in msg.tool_calls]}")
                    elif msg.content:
                        print(f"  🤖 回复: {msg.content}")
                elif isinstance(msg, ToolMessage):
                    print(f"  🔧 结果: {msg.content}")

---
## 6. 使用预构建的 ReAct Agent

LangGraph 提供了预构建的智能体，可以更快速地创建。

**ReAct** = **Re**asoning + **Act**ing（推理与行动结合）

In [ ]:
from langgraph.prebuilt import create_react_agent

# 使用预构建的 ReAct Agent（一行代码！）
# 注意：新版本 LangGraph 使用 prompt 参数代替 state_modifier
react_agent = create_react_agent(
    model=model,
    tools=tools,
    prompt="你是一个智能助手，可以使用工具帮助用户。回答要简洁准确。"
)

print("✅ ReAct Agent 创建完成！")

In [ ]:
# 测试 ReAct Agent
query = "计算一下 2 的 10 次方是多少"
print(f"👤 用户: {query}")

result = react_agent.invoke({"messages": [HumanMessage(content=query)]})

print(f"\n💬 回答: {result['messages'][-1].content}")

In [ ]:
# 测试搜索
query = "LangGraph 是什么？"
print(f"👤 用户: {query}")

result = react_agent.invoke({"messages": [HumanMessage(content=query)]})

print(f"\n💬 回答: {result['messages'][-1].content}")

---
## 7. 可视化智能体图

In [ ]:
# 获取图的 Mermaid 表示
try:
    mermaid = agent.get_graph().draw_mermaid()
    print("📊 图的 Mermaid 表示:")
    print(mermaid)
except Exception as e:
    print(f"无法生成 Mermaid: {e}")

In [ ]:
# 手动展示图的结构
print("""
📊 智能体图结构:

        ┌─────────┐
        │  START  │
        └────┬────┘
             │
             ▼
        ┌─────────┐
    ┌──▶│  agent  │◀──┐
    │   └────┬────┘   │
    │        │        │
    │        ▼        │
    │   [条件判断]     │
    │   需要工具?      │
    │        │        │
    │   ┌────┴────┐   │
    │   │  是     否  │
    │   ▼         ▼   │
    │ ┌──────┐  ┌───┐ │
    └─│tools │  │END│ │
      └──────┘  └───┘

工作流程:
1. 用户输入 → agent 节点
2. agent 决定是否需要工具
3. 如果需要 → tools 节点执行 → 返回 agent
4. 如果不需要 → 结束，返回回答
""")

---
## 8. 练习

尝试向智能体提出更复杂的问题：

In [ ]:
# 练习 1: 尝试让智能体完成多步骤任务
run_agent("现在是几点？如果每小时能写100行代码，从现在开始写到晚上10点，能写多少行？")

In [ ]:
# 练习 2: 尝试比较任务
run_agent("广州和深圳哪个城市湿度更高？高多少？")

In [ ]:
# 练习 3: 自己提一个问题试试
your_question = "请输入你的问题"  # 修改这里
run_agent(your_question)

---
## 📝 本课小结

| 概念 | 说明 |
|------|------|
| `StateGraph` | 状态图，用于定义工作流 |
| `State` | 在节点之间传递的数据 |
| `Node` | 节点，执行具体逻辑 |
| `Edge` | 边，定义节点间的连接 |
| `Conditional Edge` | 条件边，根据状态决定走向 |
| `ToolNode` | 预构建的工具执行节点 |
| `create_react_agent` | 预构建的 ReAct 智能体 |

### 智能体工作流程

```
用户输入 → agent 推理 → 需要工具? → 是 → 执行工具 → agent 推理 → ...
                              ↓
                             否 → 返回回答
```

## ➡️ 下一课

下一课我们将为智能体添加**记忆**功能，让它能够记住对话历史，实现真正的多轮对话！